# Prompt Engineering 101 - Part III.
## Pattern-Based Engineering

---

### *Software Architecture for English*

## Why Patterns?
In software engineering, a "Design Pattern" is a reusable solution to a common problem.
We apply this to AI. instead of reinventing the wheel, we use proven structures.

## The Catalog
1.  **The Persona Pattern:** "Act as X." (Sets the context/bias).
2.  **The Interview Pattern:** "Ask me questions until you know enough." (Fixes vague inputs).
3.  **The Template Pattern:** "Fill in this form." (Standardizes outputs).
4.  **The Recipe Pattern:** "I have ingredients, give me the steps." (Planning).
5.  **The Refinement Pattern:** "Critique and fix this." (Quality Control).
6.  **The Flip Pattern:** "Ask me the questions." (Testing knowledge).

---

In [ ]:
# @title 🛠️ Step 1: Laboratory Setup (Gemini API)
# We are connecting to Google's "Gemini 2.5 Flash" model.

# 1. Install the Google AI SDK
!pip install -q -U google-genai

import os
import json
import textwrap
from google.colab import userdata
import google.genai as genai
from IPython.display import display, Markdown, JSON


# 2. Configure the API Key
# Go to https://aistudio.google.com/app/apikey to get a key.
# It is free and takes 1 click.

# 3. Use Colab Secrets (Best Practice)
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# 4. Create Wrapper Class for querying
class GeminiModel:
    def __init__(self, API_KEY, model_name='gemini-2.5-flash'):
        self.client = genai.Client(api_key=API_KEY)
        self.model_name = model_name

    def generate_content(self, contents, generation_config=None):
        response = self.client.models.generate_content(
            model=self.model_name,
            contents=contents,
            config=generation_config,
        )
        return response

    def close(self):
        self.client.close()


# Free tier models have limits. In case we run out of a model quota, we'll
# use one of the fallback models.
#
# Possible model names we can use are:
# - gemini-2.5-flash-lite
# - gemini-2.5-flash
# - gemini-3-flash-preview
try:
    model = GeminiModel(GEMINI_API_KEY, model_name='gemini-2.5-flash')
    print("✅ Connection Established. The Engine is ready.")
except Exception as e:
    print(f"❌ Error: {e}. Did you paste your API key?")

### **Phase 1: Input Patterns (Gathering Intelligence)**

In [ ]:
# @title 🎭 Topic 1: The Persona Pattern (Advanced)
# We used this briefly in Mod 2. Now we treat it as a "Lens".
# A Persona is not just a voice; it is a KNOWLEDGE FILTER.

topic = "Cryptocurrency"

# Persona A: The Skeptic
prompt_a = f"""
Act as a skeptical, risk-averse Forensic Accountant.
Analyze the concept of '{topic}'. Focus on fraud risks and regulation.
"""

# Persona B: The Believer
prompt_b = f"""
Act as a visionary Web3 Venture Capitalist.
Analyze the concept of '{topic}'. Focus on future potential and disruption.
"""

print("--- THE SKEPTIC ---")
display(Markdown(model.generate_content(prompt_a).text[:500] + "...\n"))

print("--- THE BELIEVER ---")
display(Markdown(model.generate_content(prompt_b).text[:500] + "..."))

In [ ]:
# @title 🎤 Topic 2: The Interview Pattern (The "Flipped" Interaction)
# PROBLEM: You don't know what to tell the AI.
# SOLUTION: Make the AI ask YOU questions.

# This is the "God Mode" of prompting. It prevents "Garbage In".

context = "I want to launch a new coffee shop."

interview_prompt = f"""
CONTEXT: {context}

TASK:
I want you to write a marketing strategy for me.
BUT, you do not have enough information yet.
Do not generate the strategy yet.
Instead, ask me questions, one by one, to gather the info you need (Location, Target Audience, Budget).
Stop and wait for my answer after each question.
"""

# Note: In a notebook, we simulate the 'wait' by just showing the first question.
response = model.generate_content(interview_prompt).text
display(Markdown(f"**AI:** {response}"))

In [ ]:
# @title 🔄 Topic 3: The Flip Pattern (Quiz Master)
# PROBLEM: I want to test my own knowledge.
# PATTERN: "Ask me questions about X, and correct my answers."

topic = "The French Revolution"

flip_prompt = f"""
Act as a strict History Professor.
I want you to test my knowledge on {topic}.
Ask me a hard question. Wait for my answer.
Then grade me and ask the next one.
"""

display(Markdown(f"**AI:** {model.generate_content(flip_prompt).text}"))


In [ ]:
# @title 🧪 LAB 1: The Consultant Bot
# SCENARIO: You are a consultant. A client comes to you and says "My sales are down."
# That is not enough info.

# TASK: Write a prompt using the INTERVIEW PATTERN that forces the AI to act as a
# "Diagnostic Doctor" to find the root cause.

# --- STUDENT WORKSPACE ---
my_problem = "My restaurant is losing money."
my_persona = "Senior Business Analyst"

consultant_prompt = f"""
{my_persona}
{my_problem}
"""
# -------------------------

display(Markdown(model.generate_content(consultant_prompt).text))

---

### **Phase 2: Refinement Patterns (Polishing)**

In [ ]:
# @title 🧐 Topic 4: The Critique/Review Pattern
# PROBLEM: The AI's first draft is usually "average".
# PATTERN: "Generate -> Critique -> Refine".

draft_email = "Hey boss, I need a raise. I work hard. Thx."

critique_prompt = f"""
Input Text: "{draft_email}"

Task:
1. Act as a Corporate HR Director. Critique the input text for tone and persuasion.
2. List 3 specific weaknesses.
3. Do NOT rewrite it yet. Just critique.
"""

display(Markdown(model.generate_content(critique_prompt).text))

In [ ]:
# @title ✨ Topic 5: The Refinement Pattern (The Fixer)
# Continuing from above... now we apply the fix.

refinement_prompt = f"""
Based on the critique above (Tone is too casual, lacks data, unprofessional),
rewrite the email to be professional, persuasive, and data-driven.
"""

# We chain the history manually here for the demo
final_prompt = (
    critique_prompt
    + "\n"
    + model.generate_content(critique_prompt).text
    + "\n"
    + refinement_prompt
)

display(Markdown(model.generate_content(final_prompt).text))

In [ ]:
# @title 🧩 Topic 6: The Decomposition Pattern
# PROBLEM: "Write a book" is too big. The AI will hallucinate or rush.
# PATTERN: Break it down.

complex_task = "Write a comprehensive guide on how to survive in the wilderness."

decomposition_prompt = f"""
TASK: {complex_task}

Do not write the guide yet.
Instead, break this task down into a Table of Contents with 5 Chapters.
For each chapter, list 3 bullet points of key topics to cover.
"""

display(Markdown(model.generate_content(decomposition_prompt).text))

In [ ]:
# @title 😈 LAB 2: The Devil's Advocate
# SCENARIO: You are submitting a proposal to your boss. You want to know where they will attack it.
# TASK: Use the PERSONA + CRITIQUE pattern to simulate a hostile review.

# --- STUDENT WORKSPACE ---
my_proposal = "We should let all employees work from home forever to save money on office rent."

advocate_prompt = f"""
{my_proposal}
"""
# -------------------------

display(Markdown(model.generate_content(advocate_prompt).text))

---

### **Phase 3: Creation Patterns (Generating Assets)**

In [ ]:
# @title 📝 Topic 7: The Template Pattern
# PROBLEM: I need data in a specific format (e.g., a bug report, a citation).
# PATTERN: "Here is the mold. Pour the data into it."

bug_data = "The login button is broken on the mobile app when I use an iPhone."

template = """
- Title: [Short Description]
- Severity: [Low/Med/High]
- Environment: [Device/OS]
- Steps to Reproduce:
- 1. [Step 1]
- 2. [Step 2]
...
- n. [Step n]
"""

prompt = f"""
Take this data: "{bug_data}"
Fill it into this template:
{template}
"""

display(Markdown(model.generate_content(prompt).text))

In [ ]:
# @title 🍳 Topic 8: The Recipe Pattern
# PROBLEM: I know the result, I need the steps.
# PATTERN: "I want to eat X. Give me the recipe."

goal = "I want to have 10,000 followers on LinkedIn in 6 months."

recipe_prompt = f"""
GOAL: {goal}
Provide a step-by-step execution plan (Recipe) to achieve this.
Include:
- Daily habits
- Weekly milestones
- Content strategy
"""

display(Markdown(model.generate_content(recipe_prompt).text))

In [ ]:
# @title 🎨 Topic 9: The Visualization Pattern (ASCII/Mermaid)
# PROBLEM: LLMs are text-only.
# PATTERN: Ask for "Text-based visual formats" like ASCII art or Mermaid.js.

process = "A user logs in. If password is correct, go to Dashboard. If wrong, show Error."

viz_prompt = f"""
Visualize this process logic: "{process}"
Output it as a Mermaid.js Flowchart code block.
"""

display(Markdown(model.generate_content(viz_prompt).text))
# Note: You can paste the output into https://mermaid.live

---

### **Phase 4: Meta-Patterns (Advanced)**

In [ ]:
# @title 🤖 Topic 10: The Meta-Prompt (AI Writing AI)
# PROBLEM: I don't know how to write a good prompt for this task.
# PATTERN: "Write a prompt for me."

task = "I want to use AI to grade student essays."

meta_prompt = f"""
You are an expert Prompt Engineer.
I need a prompt that will allow an AI to grade 10th-grade history essays.
The prompt needs to include a Rubric, Grading Scale, and feedback instructions.

Write the prompt for me.
"""

display(Markdown(model.generate_content(meta_prompt).text))

In [ ]:
# @title 🧱 Topic 11: The Few-Shot Chain
# PROBLEM: One example isn't enough.
# PATTERN: Show the AI a pattern of "Input -> Reasoning -> Output".

# This teaches the AI *how to think*, not just what to say.
few_shot_chain = """
Input: "The car won't start."
Reasoning: The battery might be dead, or the starter is broken.
Response: "Check if the lights turn on. If not, it's the battery."

Input: "The internet is slow."
Reasoning: The router might be far away, or the ISP is having issues.
Response: "Try moving closer to the router or restarting it."

Input: "My coffee tastes burnt."
Reasoning:
"""

display(Markdown(model.generate_content(few_shot_chain).text))

In [ ]:
# @title 🧠 LAB 3: The System Builder (Capstone Prep)
# TASK: Combine 3 patterns to solve a problem.
# Scenario: You need to hire a Python Developer.

# Step 1: Use Decomposition to list the requirements.
# Step 2: Use Template to create a Job Description.
# Step 3: Use Interview Pattern to generate 5 interview questions.

# --- STUDENT WORKSPACE ---
step1_prompt = """
"""
step1_response = model.generate_content(step1_prompt).text
print("STEP1:")
display(Markdown(step1_response))

step2_prompt = f"""
{step1_response}
"""
step2_response = model.generate_content(step2_prompt).text
print("STEP2:")
display(Markdown(step2_response))

step3_prompt = f"""
{step2_response}
"""
step3_response = model.generate_content(step3_prompt).text
print("STEP3:")
display(Markdown(step3_response))
# -------------------------

---

### **Topic 12: Anti-Patterns (What to Avoid)**

*(Discussion Only)*

1. **The "Kitchen Sink" Prompt:** Trying to do too much in one prompt. (Solution: Decomposition).
2. **The "Mind Reader":** Assuming the AI knows your company context. (Solution: Persona + Context).
3. **The "Nag":** Repeating "Don't do X" 50 times. (Solution: Negative Constraints at the end).

---

## 🏠 Homework: The Socratic Tutor

### The Scenario
You are building an educational bot for a history class.
The goal is NOT to give the student the answer.
The goal is to **guide** the student to the answer by asking questions (Socratic Method).

### The Task
Write a robust prompt that:
1.  **Persona:** Acts as Socrates (wise, patient, inquisitive).
2.  **Pattern:** Uses the **Flip/Interview Pattern** (never answers, only asks).
3.  **Constraint:** If the student gets it wrong, give a hint, not the answer.
4.  **Template:** At the end of the session, output a "Report Card" in a specific format.


In [ ]:
# YOUR CODE GOES HERE